In [ ]:
import geopandas as gpd
import folium

# Load the GeoJSON
gdf = gpd.read_file("easternshore.geojson")

# Reproject to Web Mercator (EPSG:3857) for accurate centroid
gdf_proj = gdf.to_crs(epsg=3857)

# Calculate centroid in projected space
centroid_proj = gdf_proj.geometry.centroid.iloc[0]

# Transform centroid back to WGS84 (EPSG:4326) for folium
centroid_wgs = gpd.GeoSeries(centroid_proj, crs=3857).to_crs(epsg=4326).geometry.iloc[0]

# Create folium map centered at the correct location
m = folium.Map(location=[centroid_wgs.y, centroid_wgs.x], zoom_start=8)

# Add GeoJSON to the map (original data in EPSG:4326)
folium.GeoJson(gdf).add_to(m)

m

In [ ]:
tif_path = "data/sentinel_eastern_shore_2018-06-01_2018-06-30.tif"
tif_path = "data/landsat_eastern_shore_1992-03-01_1992-03-31.tif"

In [ ]:
from rasterio.windows import Window
import rasterio
import numpy as np

with rasterio.open(tif_path) as src:
    try:
        img = src.read(1)
        print(np.unique(img))
    except rasterio.errors.RasterioIOError as e:
        print(f"Failed to read block: {e}")

In [ ]:
from download_tools import get_mission, compute_ndwi

In [ ]:
mission = get_mission("sentinel-2")
mission = get_mission("landsat-5")

In [ ]:
import matplotlib.pyplot as plt

def find_non_nan_window(tif_path, bands=None, window_size=512, stride=256, threshold_ratio=0.5):
    """
    Finds a window in a raster where the data is mostly valid (not NaN or near-zero).
    
    Parameters:
        tif_path (str): Path to the raster file.
        bands (list or None): List of 1-based band indices to read and validate. If None, reads band 1.
        window_size (int): Size of the square window.
        stride (int): Step size for moving the window.
        threshold_ratio (float): Minimum proportion of valid data required.

    Returns:
        Tuple of:
            - data (np.ndarray or list of np.ndarrays): Array(s) for the selected window.
            - profile (dict): Raster profile updated for the window.
            - window (Window): The selected rasterio window.
    """
    with rasterio.open(tif_path) as src:
        width, height = src.width, src.height
        band_list = bands if bands is not None else [1]

        for y in range(0, height - window_size + 1, stride):
            for x in range(0, width - window_size + 1, stride):
                window = Window(x, y, window_size, window_size)

                # Read and validate bands
                data_list = []
                valid = True

                for b in band_list:
                    data = src.read(b, window=window)
                    data_list.append(data)

                    valid_mask = ~np.isnan(data)
                    if (
                        np.sum(valid_mask) < threshold_ratio * data.size
                        or np.sum(np.isclose(data, 0)) > (1 - threshold_ratio) * data.size
                    ):
                        valid = False
                        break  # this window is not good

                if valid:
                    print(f"Found valid window at x={x}, y={y}")
                    transform = src.window_transform(window)
                    profile = src.profile.copy()
                    profile.update({
                        "height": window.height,
                        "width": window.width,
                        "transform": transform,
                    })

                    # Return single array if only one band, else list
                    return (data_list[0] if len(data_list) == 1 else data_list), profile, window

    print("No valid window found.")
    return None, None, None




band_indices = [3, 4]  # e.g., green=3, nir=4
([green_data, nir_data], profile, window) = find_non_nan_window(tif_path, bands=band_indices, window_size=800, stride=256)
window_data = green_data

if window_data is not None:
    plt.imshow(window_data, cmap="gray")
    plt.colorbar(label="Pixel Value")
    plt.title("Non-NaN Window Preview")
    plt.axis("off")
    plt.show()

In [ ]:
plt.imshow(green_data, cmap='viridis')
plt.title("Green Band")
plt.show()

plt.imshow(nir_data, cmap='viridis')
plt.title("NIR Band")
plt.show()

In [ ]:
compute_ndwi(green_data, nir_data, profile, out_path=None, display=True)

In [ ]:
from salinity_tools import extract_salinity_features_from_mosaic

In [ ]:
extract_salinity_features_from_mosaic(
    tif_path,
    mission["band_index"],
    "data/water_mask.tif",
    "data/salinity_feature_mask.tif",
)

In [ ]:
with rasterio.open("data/water_mask.tif") as src:
    water_mask = src.read(1)
    print("Total water pixels:", np.sum(water_mask > 0))

In [ ]:
window_data, profile, window = find_non_nan_window("data/water_mask.tif", window_size=100, stride=50)

if window_data is not None:
    plt.imshow(window_data, cmap="gray")
    plt.colorbar(label="Pixel Value")
    plt.title("Non-NaN Window Preview")
    plt.axis("off")
    plt.show()

print(np.sum(window_data))

In [ ]:
window_data, profile, window = find_non_nan_window("data/salinity_feature_mask.tif", window_size=100, stride=50)

if window_data is not None:
    plt.imshow(window_data, cmap="gray")
    plt.colorbar(label="Pixel Value")
    plt.title("Non-NaN Window Preview")
    plt.axis("off")
    plt.show()

print(np.sum(window_data))